In [1]:
%%init_spark
launcher.jars = ["lsa.jar"]

In [2]:
import com.cloudera.datascience.lsa._
import com.cloudera.datascience.lsa.ParseWikipedia._
import com.cloudera.datascience.lsa.RunLSA._
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import breeze.linalg.{DenseMatrix => BDenseMatrix, DenseVector => BDenseVector, SparseVector => BSparseVector}

Intitializing Scala interpreter ...

Spark Web UI available at http://jeslava-pc:4041
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1584473017233)
SparkSession available as 'spark'


import com.cloudera.datascience.lsa._
import com.cloudera.datascience.lsa.ParseWikipedia._
import com.cloudera.datascience.lsa.RunLSA._
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import breeze.linalg.{DenseMatrix=>BDenseMatrix, DenseVector=>BDenseVector, SparseVector=>BSparseVector}


In [3]:
/*Load a set of articles from wikipedia in memory, note that this set of articles has been
previously downloaded*/
val wikixmlfile = "file:///home/jeslava/msata/cnam_RCP216/data_mining/TP5_text_data/data/enwiki.xml"
val sampleSize = 0.3
val xmlPages = ParseWikipedia.readFile(wikixmlfile, sc).sample(false, sampleSize, 11L)
xmlPages.take(2)

wikixmlfile: String = file:///home/jeslava/msata/cnam_RCP216/data_mining/TP5_text_data/data/enwiki.xml
sampleSize: Double = 0.3
xmlPages: org.apache.spark.rdd.RDD[String] = PartitionwiseSampledRDD[2] at sample at <console>:43
res0: Array[String] =
Array(<page>
    <title>AfghanistanPeople</title>
    <ns>0</ns>
    <id>15</id>
    <redirect title="Demographics of Afghanistan" />
    <revision>
      <id>616420354</id>
      <parentid>135089040</parentid>
      <timestamp>2014-07-10T19:18:43Z</timestamp>
      <contributor>
        <username>RussBot</username>
        <id>279219</id>
      </contributor>
      <minor />
      <comment>Robot: Fixing double redirect to [[Demographics of Afghanistan]]</comment>
      <model>wikitext</model>
      <format>text/x-wiki</format>
      <text xml...

In [4]:
val plainText = xmlPages.filter(_ != null).flatMap(ParseWikipedia.wikiXmlToPlainText)
plainText.take(2)

plainText: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[4] at flatMap at <console>:40
res1: Array[(String, String)] =
Array((Autism,"Autism








Autism is a neurodevelopmental disorder characterized by impaired social interaction, verbal and non-verbal communication, and restricted and repetitive behavior. Parents usually notice signs in the first two years of their child's life.  The signs typically develop gradually, but some children with autism will reach their developmental milestones at a normal pace and then regress.
Autism is highly heritable, but the cause includes both environmental factors and genetic susceptibility.  In rare cases, autism is strongly associated with agents that cause birth defects.  Controversies surround other proposed environmental caus...

In [7]:
val stopWords = sc.broadcast(ParseWikipedia
                         .loadStopWords("/home/jeslava/msata/cnam_RCP216/data_mining/TP5_text_data/deps/lsa/src/main/resources/stopwords.txt")).value
val lemmatized = plainText.mapPartitions(iter => {
           val pipeline = ParseWikipedia.createNLPPipeline();
           iter.map{ case(title, contents) =>
                         (title, ParseWikipedia.plainTextToLemmas(contents, stopWords, pipeline))};
       }).cache()

lemmatized.take(2)
val filtered = lemmatized.filter(_._2.size > 1)

stopWords: scala.collection.immutable.Set[String] = Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...

In [8]:
/*Calculate TF-IDF matrix*/
val numTerms = 1000 // nombre de termes à garder (les plus fréquents)
val (termDocMatrix, termIds, docIds, idfs) = ParseWikipedia.termDocumentMatrix(filtered, stopWords, numTerms, sc)

Number of terms: 1000


numTerms: Int = 1000
termDocMatrix: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[17] at map at ParseWikipedia.scala:64
termIds: scala.collection.Map[Int,String] = Map(645 -> improve, 892 -> identify, 69 -> court, 809 -> time, 629 -> simple, 365 -> lower, 138 -> size, 760 -> american, 101 -> agree, 479 -> child, 347 -> currently, 846 -> relatively, 909 -> film, 333 -> though, 628 -> house, 249 -> differ, 893 -> rise, 518 -> transport, 962 -> located, 468 -> october, 234 -> george, 941 -> distance, 0 -> rate, 777 -> debate, 555 -> remain, 666 -> back, 88 -> point, 481 -> company, 352 -> pay, 408 -> social, 977 -> obtain, 170 -> join, 523 -> legal, 582 -> space, 762 -> characteristic, 115 -> show, 683 -> still, 730 -> use, 217 -> description, 276 -> pet...

In [9]:
termDocMatrix.cache()

res2: termDocMatrix.type = MapPartitionsRDD[17] at map at ParseWikipedia.scala:64


In [10]:
/*Calculate LSA*/
val mat = new RowMatrix(termDocMatrix)
val k = 200 // nombre de valeurs singulières à garder
val svd = mat.computeSVD(k, computeU = true)

mat: org.apache.spark.mllib.linalg.distributed.RowMatrix = org.apache.spark.mllib.linalg.distributed.RowMatrix@1123199a
k: Int = 200
svd: org.apache.spark.mllib.linalg.SingularValueDecomposition[org.apache.spark.mllib.linalg.distributed.RowMatrix,org.apache.spark.mllib.linalg.Matrix] =
SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@750b8270,[0.9008761596036255,0.7226773744473789,0.6829274782494884,0.49254083729171805,0.40010799137301467,0.3817621668956867,0.37013433486264996,0.3609104488907344,0.35245175882849533,0.35160537076856724,0.3394133124817201,0.3369745431839872,0.33352095472304993,0.33222331184216275,0.3283206911748484,0.32148845077892735,0.3179134202958888,0.3139892824615763,0.30868416996314774,0.30585183372151775,0.3052117000251546,0.2975083905...

In [11]:
/*Show the most repeated themes and the documents where they are more present*/
val topConceptTerms = RunLSA.topTermsInTopConcepts(svd, 10, 10, termIds)
val topConceptDocs = RunLSA.topDocsInTopConcepts(svd, 10, 10, docIds)
for ((terms, docs) <- topConceptTerms.zip(topConceptDocs)) {
          println("Concept terms: " + terms.map(_._1).mkString(", "));
          println("Concept docs: " + docs.map(_._1).mkString(", "));
          println();
       }

Concept terms: code, character, computer, language, science, design, unit, law, datum, game
Concept docs: Currency code, Code, Code of Hammurabi, Code coverage, Design Science License, Control character, BLM, ASCII, List of country calling codes, Cipher

Concept terms: code, significantly, somewhat, unlike, possibly, powerful, toward, heavily, entirely, quickly
Concept docs: Currency code, Biathlon World Championships, List of Olympic medalists in biathlon, Antipope Victor IV, Alfonso IV, Aberdeen Bestiary, BLM, Cucurbitaceae, Christopher Báthory, List of anthropologists

Concept terms: unit, code, significantly, somewhat, unlike, toward, powerful, quickly, entirely, slightly
Concept docs: Astronomical Units, Currency code, Conversion of units, Unit of alcohol, Control unit, British thermal unit, Candela, Byte, Biathlon World Championships, List of Olympic medalists in biathlon

Concept terms: space, david, complete, set, real, distance, son, game, object, function
Concept docs: David 

topConceptTerms: Seq[Seq[(String, Double)]] = ArrayBuffer(ArraySeq((code,0.9945555353589379), (character,0.021694693736805312), (computer,0.019315520996154092), (language,0.017845648122745045), (science,0.016067706179285227), (design,0.015053877350772345), (unit,0.014929352494077163), (law,0.01479401486602408), (datum,0.014623734628316062), (game,0.013144710881486521)), ArraySeq((code,0.057136178940563394), (significantly,-0.0038646926388318723), (somewhat,-0.004357191678489329), (unlike,-0.004424123115154138), (possibly,-0.004568215756154709), (powerful,-0.004644885612657312), (toward,-0.004646078151510812), (heavily,-0.004655637801651445), (entirely,-0.004664205215766954), (quickly,-0.00479863011295463)), ArraySeq((unit,0.45426610702045955), (code,0.07770892827738134), (significantly,...